# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV into DataFrame

In [9]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


In [5]:
df.tail()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
77531,19100.0,11.261,own,86600,0.653580,12,2,56600,high_risk
77532,17700.0,10.662,mortgage,80900,0.629172,11,2,50900,high_risk
77533,17600.0,10.595,rent,80300,0.626401,11,2,50300,high_risk
77534,16300.0,10.068,mortgage,75300,0.601594,10,2,45300,high_risk
77535,15600.0,9.742,mortgage,72300,0.585062,9,2,42300,high_risk


In [14]:
df=df.dropna()

In [26]:
#convert interest rate to decimals


df['interest_rate'] = df['interest_rate'].astype('float') / 100

# Split the Data into Training and Testing

In [30]:
# Create our features
#based on the data, loan_status is the target which will be the y and rest are the features for X

X = df.drop(columns="loan_status")

#in the data homeowner is alpha, change that to numberic by adding columns
X = pd.get_dummies(X, columns=["homeowner"])


y = df["loan_status"].to_frame()






In [28]:
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
0,10700.0,0.07672,52800,0.431818,5,1,22800,0,1,0
1,8400.0,0.06692,43600,0.311927,3,0,13600,0,1,0
2,9000.0,0.06963,46100,0.349241,3,0,16100,0,0,1
3,10700.0,0.07664,52700,0.430740,5,1,22700,0,1,0
4,10800.0,0.07698,53000,0.433962,5,1,23000,1,0,0


In [33]:
X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,0.072923,49221.949804,0.377318,3.826610,0.392308,19221.949804,0.497472,0.398911,0.103616
std,2093.223153,0.008895,8371.635077,0.081519,1.904426,0.582086,8371.635077,0.499997,0.489678,0.304764
min,5000.000000,0.052500,30000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,0.068250,44800.000000,0.330357,3.000000,0.000000,14800.000000,0.000000,0.000000,0.000000
50%,9500.000000,0.071720,48100.000000,0.376299,4.000000,0.000000,18100.000000,0.000000,0.000000,0.000000
75%,10400.000000,0.075280,51400.000000,0.416342,4.000000,1.000000,21400.000000,1.000000,1.000000,0.000000
max,23800.000000,0.132350,105200.000000,0.714829,16.000000,3.000000,75200.000000,1.000000,1.000000,1.000000


In [17]:
print(y)

0         low_risk
1         low_risk
2         low_risk
3         low_risk
4         low_risk
5         low_risk
6         low_risk
7         low_risk
8         low_risk
9         low_risk
10        low_risk
11        low_risk
12        low_risk
13        low_risk
14        low_risk
15        low_risk
16        low_risk
17        low_risk
18        low_risk
19        low_risk
20        low_risk
21        low_risk
22        low_risk
23        low_risk
24        low_risk
25        low_risk
26        low_risk
27        low_risk
28        low_risk
29        low_risk
           ...    
77506    high_risk
77507    high_risk
77508    high_risk
77509    high_risk
77510    high_risk
77511    high_risk
77512    high_risk
77513    high_risk
77514    high_risk
77515    high_risk
77516    high_risk
77517    high_risk
77518    high_risk
77519    high_risk
77520    high_risk
77521    high_risk
77522    high_risk
77523    high_risk
77524    high_risk
77525    high_risk
77526    high_risk
77527    hig

In [31]:
# Check the balance of our target values
y["loan_status"].value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [45]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X, 
                                                   y, 
                                                   random_state=1, 
                                                   stratify=y)
X_train.shape

(58152, 10)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [46]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
data_scaler = StandardScaler()

In [47]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = data_scaler.fit(X_train)

In [48]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Simple Logistic Regression

In [49]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [50]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9543211898288821

In [51]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  571,    54],
       [   93, 18666]])

In [52]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.91      1.00      0.89      0.95      0.90       625
   low_risk       1.00      1.00      0.91      1.00      0.95      0.92     18759

avg / total       0.99      0.99      0.92      0.99      0.95      0.92     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [58]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=100)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'loan_status': 1})

In [60]:
print(y_resampled['loan_status'].value_counts())

high_risk    56277
low_risk     56277
Name: loan_status, dtype: int64


In [61]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(random_state=1)
model.fit(X_resampled, y_resampled)
y_pred = model.predict(X_test)

In [62]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.9948013433551894

In [65]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[  622,     3],
       [  105, 18654]])

In [66]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       1.00      0.99      1.00      0.99      0.99      0.99     19384



### SMOTE Oversampling

In [71]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE

X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy=0.6).fit_resample(
    X_train, y_train
)

Counter(y_resampled)


Counter({'loan_status': 1})

In [72]:
# Train the Logistic Regression model using the resampled data
model.fit(X_resampled, y_resampled)
y_pred = model.predict(X_test)

In [73]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.9932279972279972

In [74]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[  620,     5],
       [  104, 18655]])

In [79]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.99      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18759

avg / total       1.00      0.99      0.99      0.99      0.99      0.99     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [ ]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(y_resampled)

In [82]:
# Train the Logistic Regression model using the resampled data
model.fit(X_resampled, y_resampled)
y_pred = model.predict(X_test)

In [ ]:
# Calculate the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [83]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN
sm = SMOTEENN(random_state=1)
X_resampled, y_resampled = sm.fit_resample(X_train, y_train)


# View the count of target classes with Counter
Counter(y_resampled)

Counter({'loan_status': 1})

In [84]:
# Train the Logistic Regression model using the resampled data
model.fit(X_resampled, y_resampled)
y_pred = model.predict(X_test)

In [85]:
# Calculate the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.994748035609574

In [86]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[  622,     3],
       [  107, 18652]])

In [87]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       1.00      0.99      1.00      0.99      0.99      0.99     19384



RandomOverSampler# Final Questions

1. Which model had the best balanced accuracy score?

 RandomOverSampler

2. Which model had the best recall score?

    Simple Logistic Regression

3. Which model had the best geometric mean score?

    Combination Sampling